# Particle Distribution in X-direction

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.
You should consider upgrading via the '/home/katiez/FUSE/pybamm/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
coin = {"particle shape":"negative coin"}
chemistry = pybamm.parameter_sets.Ecker2015

In [3]:
# load ecker data
voltage_data_1C = pd.read_csv("pybamm/input/discharge_data/Ecker_1C.csv", header=None).to_numpy()
voltage_data_5C = pd.read_csv("pybamm/input/discharge_data/Ecker_5C.csv", header=None).to_numpy()

## Spherical

In [4]:
# choose DFN
model1 = pybamm.lithium_ion. DFN()
# choose SPMe
model2 = pybamm.lithium_ion. SPMe()

In [5]:
parameter_values0s = pybamm.ParameterValues(chemistry=chemistry)
parameter_values0s.update({"Current function [A]": "[input]"})
parameter_values0s.update({"Negative electrode diffusivity [m2.s-1]": "[function]graphite_diffusivity_Ecker2015"},
                        path="pybamm/input/parameters/lithium-ion/anodes/" + chemistry['anode'])

In [6]:
parameter_values1s = pybamm.ParameterValues(chemistry=chemistry)
parameter_values1s.update({"Current function [A]": "[input]"})
parameter_values1s.update({"Negative electrode diffusivity [m2.s-1]": "[function]graphite_diffusivity_Ecker2015_distribution"},
                        path="pybamm/input/parameters/lithium-ion/anodes/" + chemistry['anode'])
parameter_values1s.update({"Negative particle distribution in x": "[function]graphite_negative_particle_distribution_in_x_Ecker2015"},
                        path="pybamm/input/parameters/lithium-ion/anodes/" + chemistry['anode'])
#parameter_values1s.update({"Negative particle distribution in x": 2.0})
parameter_values1s

{'1 + dlnf/dlnc': 1.0,
 'Ambient temperature [K]': 298.15,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.26,
 'Cell capacity [A.h]': 0.15625,
 'Cell cooling surface area [m2]': 0.0172,
 'Cell volume [m3]': 1.61e-06,
 'Current function [A]': InputParameter(-0x34024b49b4eda613, Current function [A], children=[], domain=[], auxiliary_domains={}),
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Edge heat transfer coefficient [W.m-2.K-1]': 0.3,
 'Electrode height [m]': 0.10099999999999999,
 'Electrode width [m]': 0.085,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Ecker2015 at 0x7f0bcab4aef0>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Ecker2015 at 0x7f0bcab4ad40>,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 26120.05,
 'Initial concentration in positive electrode [mol.m-3]': 1263

In [7]:
var = pybamm.standard_spatial_vars
var_pts_s1 = {
    var.x_n: int(parameter_values0s.evaluate(model2.param.L_n / 1e-6)),
    var.x_s: int(parameter_values0s.evaluate(model2.param.L_s / 1e-6)),
    var.x_p: int(parameter_values0s.evaluate(model2.param.L_p / 1e-6)),
    var.r_n: int(parameter_values0s.evaluate(model2.param.R_n / 1e-7)),
    var.r_p: int(parameter_values0s.evaluate(model2.param.R_p / 1e-7)),
}

In [8]:
parameter_values0s.evaluate(model2.param.L_n / 1e-6)

74.0

In [9]:
sim_S0 = pybamm.Simulation(model1, parameter_values=parameter_values0s, var_pts=var_pts_s1)
sim_S1 = pybamm.Simulation(model1, parameter_values=parameter_values1s, var_pts=var_pts_s1)

In [10]:
C_rates = [1, 5]  # C-rates to solve for
capacity = parameter_values0s["Cell capacity [A.h]"]
t_evals = [
    np.linspace(0, 3800, 100), 
    np.linspace(0, 720, 100)
] # times to return the solution at
solutions_S0 = [None] * len(C_rates)  # empty list that will hold solutions

# loop over C-rates
for i, C_rate in enumerate(C_rates):
    current = C_rate * capacity
    sim_S0.solve(t_eval=t_evals[i], solver=pybamm.CasadiSolver(mode="fast"),inputs={"Current function [A]": current})
    solutions_S0[i] = sim_S0.solution

In [11]:
C_rates = [1, 5]  # C-rates to solve for
capacity = parameter_values1s["Cell capacity [A.h]"]
t_evals = [
    np.linspace(0, 3800, 100), 
    np.linspace(0, 720, 100)
] # times to return the solution at
solutions_S1 = [None] * len(C_rates)  # empty list that will hold solutions

# loop over C-rates
for i, C_rate in enumerate(C_rates):
    current = C_rate * capacity
    sim_S1.solve(t_eval=t_evals[i], solver=pybamm.CasadiSolver(mode="safe"),inputs={"Current function [A]": current})
    solutions_S1[i] = sim_S1.solution

DomainError: Domain cannot be empty if auxiliary domains are not empty

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4))

# plot the 1C Spherical results
t_sol_S0 = solutions_S0[0]["Time [s]"].entries
t_sol_S1 = solutions_S1[0]["Time [s]"].entries
ax1.plot(t_sol_S0, solutions_S0[0]["Terminal voltage [V]"](t_sol_S0), label="single", alpha=0.8, color="skyblue")
ax1.plot(t_sol_S1, solutions_S1[0]["Terminal voltage [V]"](t_sol_S1), label="distribution", linestyle="--", color="green")
ax1.plot(voltage_data_1C[:,0], voltage_data_1C[:,1], "o", color='orange')
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Voltage [V]")
ax1.set_title("1C Spherical")
ax1.legend(loc="best")

# plot the 5C Spherical results
t_sol_S0 = solutions_S0[1]["Time [s]"].entries
t_sol_S1 = solutions_S1[1]["Time [s]"].entries
ax2.plot(t_sol_S0, solutions_S0[1]["Terminal voltage [V]"](t_sol_S0), label="single", alpha=0.8, color="skyblue")
ax2.plot(t_sol_S1, solutions_S1[1]["Terminal voltage [V]"](t_sol_S1), label="distribution", linestyle="--", color="green")
ax2.plot(voltage_data_5C[:,0], voltage_data_5C[:,1], "o", color='orange')
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Voltage [V]")
ax2.set_title("5C Spherical")
ax2.legend(loc="best")

plt.tight_layout()
plt.savefig('x distribution 1')
plt.show()